# Twin-Delayed DDPG

## Importing the libraries

In [1]:
import os
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import pybullet_envs
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
from gym import wrappers
import mujoco_py

running build_ext


## Initialize the Experience Replay memory

In [2]:
class ReplayBuffer(object):

  def __init__(self, max_size=1e6):
    self.storage = []
    self.max_size = max_size
    self.ptr = 0

  def add(self, transition):
    if len(self.storage) == self.max_size:
      self.storage[int(self.ptr)] = transition
      self.ptr = (self.ptr + 1) % self.max_size
    else:
      self.storage.append(transition)

  def sample(self, batch_size):
    ind = np.random.randint(0, len(self.storage), size=batch_size)
    batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = [], [], [], [], []
    for i in ind: 
      state, next_state, action, reward, done = self.storage[i]
      batch_states.append(np.array(state, copy=False))
      batch_next_states.append(np.array(next_state, copy=False))
      batch_actions.append(np.array(action, copy=False))
      batch_rewards.append(np.array(reward, copy=False))
      batch_dones.append(np.array(done, copy=False))
    return np.array(batch_states), np.array(batch_next_states), np.array(batch_actions), np.array(batch_rewards).reshape(-1, 1), np.array(batch_dones).reshape(-1, 1)
    

## Build one neural network for the Actor model and one neural network for the Actor target

In [3]:
class Actor(nn.Module):
  
  def __init__(self, state_dim, action_dim, max_action):
    super(Actor, self).__init__()
    self.layer_1 = nn.Linear(state_dim, 400)
    self.layer_2 = nn.Linear(400, 300)
    self.layer_3 = nn.Linear(300, action_dim)
    self.max_action = max_action

  def forward(self, x):
    x = F.relu(self.layer_1(x))
    x = F.relu(self.layer_2(x))
    x = self.max_action * torch.tanh(self.layer_3(x))
    return x

## Build two neural networks for the two Critic models and two neural networks for the two Critic targets

In [4]:
class Critic(nn.Module):
  
  def __init__(self, state_dim, action_dim):
    super(Critic, self).__init__()
    # Defining the first Critic neural network
    self.layer_1 = nn.Linear(state_dim + action_dim, 400)
    self.layer_2 = nn.Linear(400, 300)
    self.layer_3 = nn.Linear(300, 1)
    # Defining the second Critic neural network
    self.layer_4 = nn.Linear(state_dim + action_dim, 400)
    self.layer_5 = nn.Linear(400, 300)
    self.layer_6 = nn.Linear(300, 1)

  def forward(self, x, u):
    xu = torch.cat([x, u], 1)
    
    x1 = F.relu(self.layer_1(xu))
    x1 = F.relu(self.layer_2(x1))
    x1 = self.layer_3(x1)
    
    x2 = F.relu(self.layer_4(xu))
    x2 = F.relu(self.layer_5(x2))
    x2 = self.layer_6(x2)

    return x1, x2

  def Q1(self, x, u):
    xu = torch.cat([x, u], 1)
    x1 = F.relu(self.layer_1(xu))
    x1 = F.relu(self.layer_2(x1))
    x1 = self.layer_3(x1)
    return x1
    


## Training Process

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class TD3(object):
  
  def __init__(self, state_dim, action_dim, max_action):
    self.actor = Actor(state_dim, action_dim, max_action).to(device)
    self.actor_target = Actor(state_dim, action_dim, max_action).to(device)
    self.actor_target.load_state_dict(self.actor.state_dict())
    self.actor_optimizer = torch.optim.Adam(self.actor.parameters())

    self.critic = Critic(state_dim, action_dim).to(device)
    self.critic_target = Critic(state_dim, action_dim).to(device)
    self.critic_target.load_state_dict(self.critic.state_dict())
    self.critic_optimizer = torch.optim.Adam(self.critic.parameters())

    self.max_action = max_action

  def select_action(self, state):
    state = torch.Tensor(state.reshape(1, -1)).to(device)
    return self.actor(state).cpu().data.numpy().flatten()

  def train(self, replay_buffer, iterations, batch_size=100, discount=0.99, tau=0.005, policy_noise=0.2, noise_clip=0.5, policy_freq=2):
    
    for it in range(iterations):
      
      batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = replay_buffer.sample(batch_size)
      state = torch.Tensor(batch_states).to(device)
      next_state = torch.Tensor(batch_next_states).to(device)
      action = torch.Tensor(batch_actions).to(device)
      reward = torch.Tensor(batch_rewards).to(device)
      done = torch.Tensor(batch_dones).to(device)
      
      next_action = self.actor_target(next_state)
      noise = torch.Tensor(batch_actions).data.normal_(0, policy_noise).to(device)
      noise = noise.clamp(-noise_clip, noise_clip)
      next_action = (next_action + noise).clamp(-self.max_action, self.max_action)
      
      target_Q1, target_Q2 = self.critic_target(next_state, next_action)
      target_Q = torch.min(target_Q1, target_Q2)
      target_Q = reward + ((1 - done) * discount * target_Q).detach()
      
      current_Q1, current_Q2 = self.critic(state, action)
      critic_loss = F.mse_loss(current_Q1, target_Q) + F.mse_loss(current_Q2, target_Q)
      
      self.critic_optimizer.zero_grad()
      critic_loss.backward()
      self.critic_optimizer.step()
      
      if it % policy_freq == 0:
        actor_loss = -self.critic.Q1(state, self.actor(state)).mean()
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
        
        for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
          target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
        
        for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
          target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
  
  def save(self, filename, directory):
    torch.save(self.actor.state_dict(), '%s/%s_actor.pth' % (directory, filename))
    torch.save(self.critic.state_dict(), '%s/%s_critic.pth' % (directory, filename))
  
  def load(self, filename, directory):
    self.actor.load_state_dict(torch.load('%s/%s_actor.pth' % (directory, filename)))
    self.critic.load_state_dict(torch.load('%s/%s_critic.pth' % (directory, filename)))


## Make a function that evaluates the policy by calculating its average reward over 10 episodes

In [1]:
def evaluate_policy(policy, eval_episodes=10, isRender=False):
  avg_reward = 0.
  for i in range(eval_episodes):
    obs = env.reset()
    done = False
    while not done:
      if i + 1 == eval_episodes and isRender:
        env.render()

      action = policy.select_action(np.array(obs))
      obs, reward, done, _ = env.step(action)
      avg_reward += reward
    
    if isRender:
      env.close()
      env.reset()

  avg_reward /= eval_episodes
  print ("---------------------------------------")
  print ("Average Reward over the Evaluation Step: %f" % (avg_reward))
  print ("---------------------------------------")
  return avg_reward


## Set the parameters

In [9]:
env_name = "Ant-v2"
seed = 0
start_timesteps = 1e4
eval_freq = 5e3
max_timesteps = 1e6
save_models = True
expl_noise = 0.1
batch_size = 100
discount = 0.99 
tau = 0.005
policy_noise = 0.2
noise_clip = 0.5
policy_freq = 2

## Create a file name for the two saved models: the Actor and Critic models

In [10]:
file_name = "%s_%s_%s" % ("TD3", env_name, str(seed))
print ("---------------------------------------")
print ("Settings: %s" % (file_name))
print ("---------------------------------------")

---------------------------------------
Settings: TD3_Ant-v2_0
---------------------------------------


## Create a folder inside which will be saved the trained models

In [11]:
if not os.path.exists("./results"):
  os.makedirs("./results")
  
if save_models and not os.path.exists("./pytorch_models"):
  os.makedirs("./pytorch_models")


## Create the PyBullet environment

In [12]:
env = gym.make(env_name)

## Set seeds and we get the necessary information on the states and actions in the chosen environment

In [13]:
env.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
max_action = float(env.action_space.high[0])

## Create the policy network and replay buffer

In [14]:
# Policy network:
policy = TD3(state_dim, action_dim, max_action)

# Replay buffer
replay_buffer = ReplayBuffer()


## Define a list where all the evaluation results over 10 episodes are stored

In [19]:
evaluations = [evaluate_policy(policy, 2, True)]

Creating window glfw
---------------------------------------
Average Reward over the Evaluation Step: 985.153252
---------------------------------------


## Create a new folder directory in which the final results (videos of the agent) will be populated

In [17]:
def mkdir(base, name):
    path = os.path.join(base, name)
    if not os.path.exists(path):
        os.makedirs(path)
    return path

work_dir = mkdir('exp', 'brs')
monitor_dir = mkdir(work_dir, 'monitor')
max_episode_steps = env._max_episode_steps


## Training

In [19]:
total_timesteps = 0
timesteps_since_eval = 0
episode_num = 0
done = True
t0 = time.time()

while total_timesteps < max_timesteps:
  
  # If the episode is done
  if done:
    if total_timesteps != 0:
      print("Total Timesteps: {} Episode Num: {} Reward: {}".format(total_timesteps, episode_num, episode_reward))
      policy.train(replay_buffer, episode_timesteps, batch_size, discount, tau, policy_noise, noise_clip, policy_freq)

    if timesteps_since_eval >= eval_freq:
      timesteps_since_eval %= eval_freq
      evaluations.append(evaluate_policy(policy))
      policy.save(file_name, directory="./pytorch_models")
      np.save("./results/%s" % (file_name), evaluations)
    
    obs = env.reset()
    done = False
    episode_reward = 0
    episode_timesteps = 0
    episode_num += 1
  
  # Before 10000 timesteps, we play random actions
  if total_timesteps < start_timesteps:
    action = env.action_space.sample()
  else:
    action = policy.select_action(np.array(obs))
    if expl_noise != 0:
      action = (action + np.random.normal(0, expl_noise, size=env.action_space.shape[0])).clip(env.action_space.low, env.action_space.high)
  
  new_obs, reward, done, _ = env.step(action)

  done_bool = 0 if episode_timesteps + 1 == env._max_episode_steps else float(done)
  episode_reward += reward
  replay_buffer.add((obs, new_obs, action, reward, done_bool))

  obs = new_obs
  episode_timesteps += 1
  total_timesteps += 1
  timesteps_since_eval += 1

# Add the last policy evaluation to our list of evaluations and we save our model
evaluations.append(evaluate_policy(policy))
if save_models: policy.save("%s" % (file_name), directory="./pytorch_models")
np.save("./results/%s" % (file_name), evaluations)


Total Timesteps: 157 Episode Num: 1 Reward: -143.54351621097098
Total Timesteps: 236 Episode Num: 2 Reward: -55.316183680573666
Total Timesteps: 1236 Episode Num: 3 Reward: -339.29842315590344
Total Timesteps: 1504 Episode Num: 4 Reward: -97.77895997431226
Total Timesteps: 2504 Episode Num: 5 Reward: -399.7608979445045
Total Timesteps: 2559 Episode Num: 6 Reward: -31.109622138509856
Total Timesteps: 2595 Episode Num: 7 Reward: -8.118218120428118
Total Timesteps: 2631 Episode Num: 8 Reward: -2.60210228747282
Total Timesteps: 3631 Episode Num: 9 Reward: -323.2135853339898
Total Timesteps: 3656 Episode Num: 10 Reward: -0.8087687788968088
Total Timesteps: 3764 Episode Num: 11 Reward: -24.059611080985537
Total Timesteps: 3881 Episode Num: 12 Reward: -88.70078300295349
Total Timesteps: 4881 Episode Num: 13 Reward: -359.119997335086
Total Timesteps: 4909 Episode Num: 14 Reward: -26.255804837668922
Total Timesteps: 4948 Episode Num: 15 Reward: -42.701436977400284
Total Timesteps: 5948 Episode 

## Inference

In [16]:
file_name = "%s_%s_%s" % ("TD3", env_name, str(seed))
print ("---------------------------------------")
print ("Settings: %s" % (file_name))
print ("---------------------------------------")

eval_episodes = 10
isRender = True
env = gym.make(env_name)
max_episode_steps = env._max_episode_steps
env.reset()

env.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)

state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
max_action = float(env.action_space.high[0])

policy = TD3(state_dim, action_dim, max_action)
policy.load(file_name, './pytorch_models/')
_ = evaluate_policy(policy, eval_episodes=eval_episodes, isRender=isRender)


---------------------------------------
Settings: TD3_Ant-v2_0
---------------------------------------
Creating window glfw
---------------------------------------
Average Reward over the Evaluation Step: 5298.986548
---------------------------------------
